In [1]:
import folium
import pandas as pd
import re

In [2]:
# support de réprésentaion géographique des données 
# centrée autour de Marseille de coordonnées gps : [43.3, 5.4]
m = folium.Map(location=[43.3, 5.4])
m

In [3]:
# recupération des données scrappé le 8 octobre 2021 sur superimmo.com
df = pd.read_csv('../datas/superimmo/20211008/superimmo_20211008_13.csv')

# repartition des annonces par ville (code postal en fait)
df.location.value_counts()

Aix-en-Provence (13100)        592
Marseille 8ème (13008)         458
Marseille 9ème (13009)         396
Marseille 13ème (13013)        382
Martigues (13500)              328
                              ... 
L Estaque (13016)                1
SAINT-ESTEVE-JANSON (13610)      1
ARLES (13200)                    1
Gardanne (13120) - Biver         1
Martigues (13117)                1
Name: location, Length: 186, dtype: int64

In [4]:
#extraction du code postaux des "location"(exemple:Marseille 8ème (13008)) via une regex
def getcp(location):
    result = re.search(r".*\((.*)\)", location)
    if result:
        return result.group(1)
    

df['Code_postal'] = df.location.apply(lambda x: getcp(str(x)))
# df.info()
# ça n'a pas marché partout
df[(df['Code_postal'].isna())][['location','Code_postal','titre']]
# parce que location n'y ait pas partout ...

# on va les retirer pour le moment mais il ne faut pas qu'on les oublie ...
df = df[-(df['Code_postal'].isna())]

# df.info()

In [5]:
# obtenir les coordonnées gps des codes postaux obtenus
# à partir du fichier pris sur : 
# https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/
coo=pd.read_csv('../datas/locations/laposte_hexasmal.csv',sep=";")
#coo.info()

# formatage
coo.Code_postal = coo.Code_postal.astype('str')

### ici je voulais faire une jointure entre les deux fichiers 
### un truc dans le genre : 
### df['gps'] = pd.merge(df, coo, how="left", on=["Code_postal"])
### mais ça ne marchait pas bien 
### 
### J'ai finalement constuit un dictionnaire clé->valeur entre les codes postaux et le coo gps
### il y surement moyen de faire mieux 

cpcoo={}
for cp,coo in zip (coo['Code_postal'].to_list(),coo['coordonnees_gps'].to_list()) :
    cpcoo[cp]=coo

def getcoo(x,cpcoo):       
    try :
        return cpcoo[x];
    except :
         return'0,0';
    

df['gps']=df.Code_postal.apply(lambda x: getcoo(str(x),cpcoo))
print(df[['Code_postal','gps']])


     Code_postal                          gps
0          13570   43.894724982,4.74934042876
1          13270  43.4556812107,4.90452133912
2          13006  43.2870989546,5.38104785394
3          13001  43.2999009436,5.38227869795
4          13013  43.3492780005,5.43372238738
...          ...                          ...
9350       13320  43.4472296617,5.41237056144
9351          13                          0,0
9353       13010  43.2755155435,5.42681101081
9354       13400  43.2934843764,5.56331273477
9355       13240   43.3934944571,5.3808040555

[9188 rows x 2 columns]


In [6]:
# calcul du prix au metre carrée par code postal
df = df[(df.type=='appartement')]

# seulement si on a biens des valeurs 
df = df[-(df.prix.isna())]
df = df[-(df.surface.isna())]

# formatage
# 120 500 € => 120000 par exemple
df.prix = df.prix.apply(lambda x: int(float(str(x)
                            .replace('€','')
                            .replace(' ','')
                            .replace(',','.') # certaines annonces ont des prix à virgules
                        )))
df.prix = df.prix.astype(int)

# 85,65 => 86.65 par exemple
df.surface = df.surface.apply(lambda x: str(x).replace(' ','').replace(',','.'))
df.surface = df.surface.astype(float)

# deduction du prix au metre carré
df['pm2']=df['prix']/df['surface']

print(df.groupby('Code_postal').describe()['pm2'])



             count         mean          std          min          25%  \
Code_postal                                                              
13            14.0  2720.270798  1535.387533  1494.413408  1761.746715   
13001         89.0  3621.303658  1079.503533  1673.684211  2838.862559   
13002        107.0  3650.382593  1355.159108  1333.333333  2929.669031   
13003        193.0  2362.479812   800.539667  1037.037037  1742.424242   
13004        150.0  2915.846770   700.284150  1508.474576  2422.670639   
...            ...          ...          ...          ...          ...   
13920          4.0  3193.762363   957.940412  2450.000000  2489.314516   
13940          1.0  3249.391304          NaN  3249.391304  3249.391304   
13960         30.0  6402.629602  1467.392690  3795.046242  5415.221941   
13980          4.0  4437.905844   703.896161  3946.428571  4040.178571   
13990          3.0  3427.032227   271.566942  3114.285714  3338.961039   

                     50%          75%

In [7]:
# recensement des données à placer sur la carte regroupées par code postal
dfByCp = df.groupby('Code_postal').agg({'pm2' : ['mean'],     # prix au mettre moyen des annonces
                                        'gps':['min'],        # coordonnées gps (qui sont normalement toutes les mêmes)
                                        'location':['min']    # nom de lieu correspondant au code postal
                                       })
print(dfByCp)

print(dfByCp['pm2']['mean'].min())

                     pm2                          gps  \
                    mean                          min   
Code_postal                                             
13           2720.270798                          0,0   
13001        3621.303658  43.2999009436,5.38227869795   
13002        3650.382593  43.3126964178,5.36364983265   
13003        2362.479812  43.3121200046,5.38010981423   
13004        2915.846770   43.306733355,5.40087488752   
...                  ...                          ...   
13920        3193.762363  43.4554029916,5.01494725455   
13940        3249.391304   43.8053410404,4.9488694514   
13960        6402.629602  43.3456690305,5.11765506999   
13980        4437.905844  43.7074197745,5.15105029207   
13990        3427.032227  43.7232610704,4.72310168882   

                                     location  
                                          min  
Code_postal                                    
13                       Aix-en-Provence (13)  
13001    

In [8]:
# utilisation d'un dégradé de couleur 
# pour indentifier visuellement les prix au mettre carré
# obtenue grace au site : https://colordesigner.io/gradient-generator

gradient40 = ['#fafa6e',
'#faf568',
'#f9f063',
'#f9eb5d',
'#f8e658',
'#f8e153',
'#f7dc4d',
'#f7d748',
'#f6d243',
'#f6cd3f',
'#f5c83a',
'#f4c335',
'#f4bd30',
'#f3b82c',
'#f2b327',
'#f1ae23',
'#f0a91f',
'#efa41b',
'#ee9e16',
'#ed9912',
'#ec940e',
'#eb8e0a',
'#ea8906',
'#e88303',
'#e77e01',
'#e67800',
'#e47300',
'#e36d00',
'#e16700',
'#df6100',
'#de5b00',
'#dc5501',
'#da4f02',
'#d84804',
'#d64105',
'#d43a07',
'#d23209',
'#d0290b',
'#cd1e0d',
'#cb0f0f']

# colorimétrie faite en fractionnant les valeurs 
# sur une échelle de 40 valeurs partagées 
# entre le prix au mettre carré minimum et maximum
mrg=1 # marge pour éviter les sorties mémoires "out of index"
print("Répartion géographique des prix au mettre carré")
print("des appartements des annonces de superimmo.com")
print("des bouches du rhone fait le 8 octobre 2021")
print("min : "+str(dfByCp['pm2']['mean'].min()))
print("max : "+str(dfByCp['pm2']['mean'].max()))

for gps,location,pm2 in zip (dfByCp['gps']['min'].to_list(),
                             dfByCp['location']['min'].to_list(),
                             dfByCp['pm2']['mean'].to_list()) :
    folium.CircleMarker(
    location=gps.split(','),
    radius=10,
    popup=location+" : "+str(pm2),
    color=gradient40[int((pm2-dfByCp['pm2']['mean'].min()-mrg)/(dfByCp['pm2']['mean'].max()+100-dfByCp['pm2']['mean'].min()-mrg)*40)], 
    fill=True,
    fill_color=gradient40[int((pm2-dfByCp['pm2']['mean'].min()-mrg)/(dfByCp['pm2']['mean'].max()+100-dfByCp['pm2']['mean'].min()-mrg)*40)],
    ).add_to(m)    
m 

Répartion géographique des prix au mettre carré
des appartements des annonces de superimmo.com
des bouches du rhone fait le 8 octobre 2021
min : 1585.6974438352806
max : 8089.929325984766
